# Greedy Feature Selection
Author: Amir H. Sadeghi
Last Update: 04/10/2023

## Import Libraries

In [32]:
import pandas as pd
import numpy as np
from scipy.stats import bernoulli
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import entropy


## Read Data

In [33]:
df = pd.read_csv("GenData_LinReg.csv")
m = df.shape[0]
numberOfFeatures = df.shape[1] - 1
df.head()

,x1,x2,x3,corr1,corr2,noise1,noise2,noise3,noise4,noise5,noise6,noise7,noise8,noise9,noise10,y
0,1.439524,3.579187,9.596431,-0.020631,-101.159764,4.338788,2.464870,7.115856,2.837020,2.449958,1.731931,3.032334,4.524291,3.909282,3.213687,28.539186
1,1.769823,5.513767,6.937239,0.683886,-212.185187,3.583505,4.068732,5.230756,2.795022,1.317801,0.865130,2.939818,1.975339,4.422803,2.184427,26.731980
2,3.558708,4.506616,2.204565,-1.620440,-259.446800,4.695768,2.966538,4.796146,0.318664,4.562151,1.712156,2.310784,3.518887,2.675007,3.619724,24.832210
3,2.070508,4.304915,4.629582,-0.327478,-162.221712,4.146469,3.032776,2.446522,2.685872,4.596986,-0.258177,3.006083,3.024128,4.650399,4.298468,21.811595
4,2.129288,3.096763,1.756980,0.822618,-109.831568,4.598500,3.658609,3.389158,4.046742,0.546048,-0.638760,1.507549,2.028737,4.151604,4.274389,16.382843


,x1,x2,x3,corr1,corr2,noise1,noise2,noise3,noise4,noise5,noise6,noise7,noise8,noise9,noise10,y
0,1.439524,3.579187,9.596431,-0.020631,-101.159764,4.338788,2.464870,7.115856,2.837020,2.449958,1.731931,3.032334,4.524291,3.909282,3.213687,28.539186
1,1.769823,5.513767,6.937239,0.683886,-212.185187,3.583505,4.068732,5.230756,2.795022,1.317801,0.865130,2.939818,1.975339,4.422803,2.184427,26.731980
2,3.558708,4.506616,2.204565,-1.620440,-259.446800,4.695768,2.966538,4.796146,0.318664,4.562151,1.712156,2.310784,3.518887,2.675007,3.619724,24.832210
3,2.070508,4.304915,4.629582,-0.327478,-162.221712,4.146469,3.032776,2.446522,2.685872,4.596986,-0.258177,3.006083,3.024128,4.650399,4.298468,21.811595
4,2.129288,3.096763,1.756980,0.822618,-109.831568,4.598500,3.658609,3.389158,4.046742,0.546048,-0.638760,1.507549,2.028737,4.151604,4.274389,16.382843


In [34]:
# Calculate the entropy of each column
entropies = df.apply(lambda x: entropy(x.value_counts(normalize=True)), axis=0)
entropies

x1         4.60517
x2         4.60517
x3         4.60517
corr1      4.60517
corr2      4.60517
noise1     4.60517
noise2     4.60517
noise3     4.60517
noise4     4.60517
noise5     4.60517
noise6     4.60517
noise7     4.60517
noise8     4.60517
noise9     4.60517
noise10    4.60517
y          4.60517
dtype: float64

x1         4.60517
x2         4.60517
x3         4.60517
corr1      4.60517
corr2      4.60517
noise1     4.60517
noise2     4.60517
noise3     4.60517
noise4     4.60517
noise5     4.60517
noise6     4.60517
noise7     4.60517
noise8     4.60517
noise9     4.60517
noise10    4.60517
y          4.60517
dtype: float64

In [35]:
def solutionGenerator(n,probVector):
    solutionList = []
    for i in range(0,n):
        solution = []
        for j in range(0,len(probVector)):
            solution.append(bernoulli(probVector[j]).rvs())
        #print(solution)
        solutionList.append(solution)
    return solutionList

In [36]:
def modelFit(solutionList):
    df = pd.read_csv("GenData_LinReg.csv")
    mseList = []
    for sol in solutionList:
        X_train, X_test, y_train, y_test = train_test_split(df[['x1', 'x2', 'x3', 'corr1', 'corr2', 'noise1', 'noise2', 'noise3',
       'noise4', 'noise5', 'noise6', 'noise7', 'noise8', 'noise9', 'noise10']], df['y'], test_size=0.3, random_state=42)

        # Convert binary list to list of column indices
        keep_cols = [i for i, x in enumerate(sol) if x]

        # Use the `iloc` indexing function to select columns based on the list of column indices
        X_train = X_train.iloc[:, keep_cols]
        X_test = X_test.iloc[:, keep_cols]

        # Create linear regression object
        model = LinearRegression()

        # Fit the model using the training data
        model.fit(X_train, y_train)

        # Make predictions on the test data
        y_pred = model.predict(X_test)

        # Calculate the mean squared error and R-squared on the test data
        mse = mean_squared_error(y_test, y_pred)
        mseList.append(mse)
    performance = sum(mseList) / len(mseList)
    return performance

# Example for Presentation on April 28th

In [37]:
p = [0.5] * numberOfFeatures
p[1] = 1
p[0] = 1
#p[4] = 1
#print("P: ", p)
solList = solutionGenerator(10, p)

mse = modelFit(solList)
print("avg mse: ", mse/(10**7))

avg mse:  1.7442987931872278


In [38]:
solList

[[1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
 [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
 [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1],
 [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
 [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1],
 [1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]]

In [39]:
sum_list = [sum(col) for col in zip(*solList)]
sum_list

[10, 10, 2, 3, 6, 5, 3, 6, 7, 4, 5, 4, 3, 4, 6]

In [40]:
min_value = min(sum_list)
max_value = max(sum_list)

portion_list = [(x - min_value) / (max_value - min_value) for x in sum_list]
new_p=[0.5]*len(portion_list)
for n in range(0,len(portion_list)):
    new_p[n] = min(1,p[n]*(1+portion_list[n]))
new_p

[1,
 1,
 0.5,
 0.5625,
 0.75,
 0.6875,
 0.5625,
 0.75,
 0.8125,
 0.625,
 0.6875,
 0.625,
 0.5625,
 0.625,
 0.75]

In [41]:
p = [0.5] * numberOfFeatures
p[0] = 0
p[1] = 1
print("P: ", p)
solList = solutionGenerator(10, p)

mse = modelFit(solList)
print("avg mse: ", mse/(10**7))

P:  [0, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
avg mse:  0.37225051488857874


In [42]:
p = [0.5] * numberOfFeatures
p[0] = 0
p[1] = 0
print("P: ", p)
solList = solutionGenerator(10, p)

mse = modelFit(solList)
print("avg mse: ", mse/(10**7))

P:  [0, 0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
avg mse:  0.5112131004744824


In [43]:
solList

[[0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1],
 [0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1],
 [0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
 [0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0]]

In [44]:
#given branch x1=0
p = [0.5] * numberOfFeatures
def updateP(p):
    k_max = 1
    for k in range(0,k_max):
        for i in range(0,numberOfFeatures):
            for j in range(0,numberOfFeatures):
                #print("===================")
                p_i_out = [1]*numberOfFeatures
                p_i_out[i] = 0
                i_out = solutionGenerator(10, p_i_out)

                p_j_out = [1]*numberOfFeatures
                p_j_out[j] = 0
                j_out = solutionGenerator(10, p_j_out)


                performance_i_out = modelFit(i_out)
                performance_j_out = modelFit(j_out)
                if performance_i_out >= performance_j_out:
                    p[i] *= 1.1
                    p[j] /= 1.1
                #print("i=",i,"j=",j)
                #print("--")
                #print(p)
        #print("Finished!: ",p)
    return p

# print
updateP(p)

[1.5692141883605017,
 0.23325369010486655,
 1.8987491679162074,
 0.13166562715303984,
 0.6050000000000001,
 0.7320500000000003,
 0.5,
 0.8857805000000005,
 1.0717944050000008,
 1.2968712300500012,
 0.28223696502688855,
 0.4132231404958676,
 0.15931540885517823,
 0.34150672768253515,
 0.19277164471476568]

[1.5692141883605017,
 0.23325369010486655,
 1.8987491679162074,
 0.13166562715303984,
 0.6050000000000001,
 0.7320500000000003,
 0.5,
 0.8857805000000005,
 1.0717944050000008,
 1.2968712300500012,
 0.28223696502688855,
 0.4132231404958676,
 0.15931540885517823,
 0.34150672768253515,
 0.19277164471476568]

In [45]:
solutionGenerator(5,[0.9,0.0,0.2])

[[1, 0, 1], [0, 0, 0], [1, 0, 0], [1, 0, 1], [1, 0, 0]]

[[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 1]]

In [46]:
#main code
p = [0.5] * numberOfFeatures

includeFeature = []
notIncludeFeature = []
SurroundingFeature = []

treeLevel = 1
# BUG: The a1 is changing
pInclude = p.copy()
pInclude[treeLevel] = 1
solutionInclude = solutionGenerator(10, pInclude)

pNotInclude = p.copy()
pNotInclude[treeLevel] = 0
solutionNotInclude = solutionGenerator(10, pNotInclude)

# BUG: THE PARTITIONS ---> INCLUDE/EXCLUDE
pSurrounding = p.copy()
pSurrounding[treeLevel-1] = 1
solutionSurrounding = solutionGenerator(10, pSurrounding)

mseInclude = modelFit(solutionInclude)

mseNotInclude = modelFit(solutionNotInclude)

if mseInclude < mseNotInclude:
    p[treeLevel] = min(p[treeLevel]*1.01,1)
else:
    p[treeLevel] = min(p[treeLevel]/1.01,1)

#if (mseInclude+mseNotInclude)/2:
#    print("Need Backtrack")

In [47]:
mseInclude

43065207.41348679

31330928.288732618

In [48]:
mseNotInclude

47559754.00600127

39063103.439355835

In [49]:
p

[0.5, 0.505, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

[0.5, 0.505, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [50]:
solutionInclude

[[0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1],
 [1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
 [1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0],
 [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0],
 [0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0]]

[[0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0],
 [1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
 [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0]]